In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef('Series')]
[<class 'str'>, <class 'float'>, <class 'bool'>]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta')]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval')]
[<class 'str'>, <class 'float'>, <class 'bool'>, ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval'), <class 'numpy.datetime64'>, <class 'numpy.timedelta64'>, <class 'datetime.datetime'>]
[ForwardRef('Timestamp'), <class 'datetime.datetime'>, <class 'numpy.datetime64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[ForwardRef('Timedelta'), <class 'datetime.timedelta'>, <class 'numpy.timedelta64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[<class 'str'>, <class 'datetime.tzinfo'>]
[<class 'str'>, <class 'int'>]
[typing.Hashable, typing.Sequence[typing.Ha

[15:49:28] Initializing Normalizer


READ ME

This file is used to create the CNS database to validate the combined subMD, and subFP using models.



In [2]:
cns_raw =  pd.read_csv('DrugBank_parsed.tsv', delimiter= '\t')

In [3]:
cns_raw.head()

,type,drugbank_id,name,description,groups,atc_codes,categories,smiles,inchi,inchikey,aliases
0,biotech,DB00001,Lepirudin,Lepirudin is a recombinant hirudin formed by 6...,approved|withdrawn,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",NaN,NaN,NaN,Lepirudin|Refludan
1,biotech,DB00002,Cetuximab,Cetuximab is a recombinant chimeric human/mous...,approved,L01FE01,"Amino Acids, Peptides, and Proteins|Antibodies...",NaN,NaN,NaN,Cetuximab|Erbitux
2,biotech,DB00003,Dornase alfa,Dornase alfa is a biosynthetic form of human d...,approved,R05CB13,"Amino Acids, Peptides, and Proteins|Cough and ...",NaN,NaN,NaN,|Dornase alfa|Pulmozyme|Pulmozyme 1mg/ml
3,biotech,DB00004,Denileukin diftitox,A recombinant DNA-derived cytotoxic protein co...,approved|investigational,L01XX29,"ADP Ribose Transferases|Amino Acids, Peptides,...",NaN,NaN,NaN,Denileukin diftitox|Ontak
4,biotech,DB00005,Etanercept,Dimeric fusion protein consisting of the extra...,approved|investigational,L04AB01,"Agents reducing cytokine levels|Amino Acids, P...",NaN,NaN,NaN,|Benepali|Brenzys|Enbrel|Erelzi|Etanercept|E...


In [4]:
#Remove if no smiles is available

cns_smiles = cns_raw[((cns_raw['smiles'].notna()))]
cns_smiles = cns_smiles.reset_index(drop=True)
print(f'Length with SMILES: {len(cns_smiles)}')


Length with SMILES: 11583


In [5]:
#Remove rows where categories is missing
cns_cat = cns_smiles[((cns_smiles['categories'].notna()))]
cns_cat = cns_cat.reset_index(drop=True)
print(f'Length without missing categories: {len(cns_cat)}')

Length without missing categories: 7044


In [6]:
#Find CNS targeting drugs: create new column 'target'
keywords = ['CNS', 'Central Nervous System']
cns_cat['status_cns'] = cns_cat['categories'].apply(lambda x: 'CNS' if any(keyword.lower() in str(x).lower() for keyword in keywords) else 'Non-CNS')

In [7]:
#Keep the relevant column

cns = cns_cat[['smiles','status_cns']]
print(len(cns))
cns.head()

cns = cns.rename(columns={'smiles': 'SMILES_raw'})


7044


In [8]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [9]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [10]:
def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [11]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_cns'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(len(df))
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    #df['inchi_stereo'].values_counts

    return df

In [12]:


df_sd = standardize_workflow(cns)
df_sd.to_csv('cns_standardized_val.csv')

df_valid = missing_inchi(df_sd)
df_valid.to_csv('cns_have_inchi_key_val.csv')

df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_connectivity_inchi.to_csv('cns_connectivity_inchi_val.csv')

df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_no_duplicates.to_csv('cns_no_duplicates_val.csv')

An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:49:51] WARNING: Omitted undefined stereo

[15:49:51] WARNING: Proton(s) added/removed



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:49:52] WARNING: Omitted undefined stereo

[15:49:52] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

...

[15:54:21] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:54:22] WARNING: Charges were rearranged

[15:54:22] WARNING: Omitted undefined stereo

[15:54:22] WARNING: Charges were rearranged



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
df length after standardization: 7044
DB length: 7044,        SMILES nan: 1839,        inchi key nan: 1839
-----remove missing inchikey----
updated length: 5205
length: 5205
unique_inchi: 5071
Contradicting duplicates: 13
5175
5058


C:\Users\Lily\AppData\Local\Temp\ipykernel_17396\2591752290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)


In [13]:
df_no_duplicates.head()

,SMILES_raw,status_cns,papyrus_SMILES,papyrus_inchi_key,inchi_connectivity
0,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O,Non-CNS,Cc1ncc(COP(=O)(O)O)c(C=O)c1O,NGVDGCNFYWLIFO-UHFFFAOYSA-N,NGVDGCNFYWLIFO
1,NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...,Non-CNS,Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)...,MSTNYGQPCMXVAQ-UHFFFAOYSA-N,MSTNYGQPCMXVAQ
2,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...,Non-CNS,C[S+](CCC(N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(N)nc...,MEFKEPWMEQBLKI-YDBXVIPQSA-O,MEFKEPWMEQBLKI
3,[H][C@]12CS[C@@H](CCCCC(O)=O)[C@@]1([H])NC(=O)N2,Non-CNS,O=C(O)CCCC[C@@H]1SC[C@@H]2NC(=O)N[C@@H]21,YBJHBAHKTGYVGT-ZKWXMUAHSA-N,YBJHBAHKTGYVGT
4,NCCCNCCCCNCCCN,Non-CNS,NCCCNCCCCNCCCN,PFNFFQXMRSDOHW-UHFFFAOYSA-N,PFNFFQXMRSDOHW


Remove training molecules

In [14]:
influx = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/influx/combined_influx_train_raw.csv')
efflux = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/efflux/combined_efflux_train_raw.csv')
pampa = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/pampa/combined_pampa_train_raw.csv')
bbb = pd.read_csv('../../../3_Combined/31_Combined_datasets/1_data_curation/bbb/combined_bbb_train_raw.csv')

In [15]:
mask = df_no_duplicates['inchi_connectivity'].isin(influx['inchi_connectivity'])
common_values_list = df_no_duplicates[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = df_no_duplicates[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')

Common molecules 175
After removed: 4883


In [45]:
cns_val.head()

,SMILES_raw,status_cns,papyrus_SMILES,papyrus_inchi_key,inchi_connectivity
0,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O,Non-CNS,Cc1ncc(COP(=O)(O)O)c(C=O)c1O,NGVDGCNFYWLIFO-UHFFFAOYSA-N,NGVDGCNFYWLIFO
1,NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...,Non-CNS,Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)...,MSTNYGQPCMXVAQ-UHFFFAOYSA-N,MSTNYGQPCMXVAQ
2,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...,Non-CNS,C[S+](CCC(N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(N)nc...,MEFKEPWMEQBLKI-YDBXVIPQSA-O,MEFKEPWMEQBLKI
3,[H][C@]12CS[C@@H](CCCCC(O)=O)[C@@]1([H])NC(=O)N2,Non-CNS,O=C(O)CCCC[C@@H]1SC[C@@H]2NC(=O)N[C@@H]21,YBJHBAHKTGYVGT-ZKWXMUAHSA-N,YBJHBAHKTGYVGT
4,NCCCNCCCCNCCCN,Non-CNS,NCCCNCCCCNCCCN,PFNFFQXMRSDOHW-UHFFFAOYSA-N,PFNFFQXMRSDOHW


In [16]:
mask = cns_val['inchi_connectivity'].isin(efflux['inchi_connectivity'])
common_values_list = cns_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = cns_val[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')


Common molecules 480
After removed: 4403


In [17]:
mask = cns_val['inchi_connectivity'].isin(pampa['inchi_connectivity'])
common_values_list = cns_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = cns_val[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')

Common molecules 126
After removed: 4277


In [18]:
mask = cns_val['inchi_connectivity'].isin(bbb['inchi_connectivity'])
common_values_list = cns_val[mask]['inchi_connectivity'].tolist()
print(f'Common molecules {len(common_values_list)}')
cns_val = cns_val[~mask]
cns_val = cns_val.reset_index(drop=True)
print(f'After removed: {len(cns_val)}')

cns_val.to_csv('cns_validation.csv')

Common molecules 664
After removed: 3613
